# yul moldauer and curran phillips in Parallel Bars

In [1]:
# import sys
# sys.executable


In [2]:
# /Users/ryantalbot/opt/anaconda3/envs/tf2/bin/python -m pip install lightgbm

In [3]:
# import threadpoolctl
# print(threadpoolctl.__version__)


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [2]:
df = pd.read_csv('../../Combine_Data/men/pb_encoded.csv')
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_pb.csv')
pbnames = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_pbnames.csv')
pbolymp = pd.read_csv('../../Data/cleandata22-23/men22_23.csv')


In [42]:
olymp

,Rank,D,E,ND,Total,year,Name,Nation,round_final,round_qual,ypred
0,23.0,3.9,6.900,0.0,10.800,2022,236,61,False,True,0
1,23.0,3.9,6.900,0.0,10.800,2022,236,61,False,True,0
2,54.0,4.4,8.066,0.0,12.466,2023,236,61,False,True,0
3,12.0,5.8,8.400,0.0,14.200,2022,531,26,False,False,0
4,26.0,5.8,8.500,0.0,14.300,2022,531,26,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...
2369,42.0,4.3,6.400,0.0,10.700,2023,453,56,False,True,0
2370,15.0,6.6,6.950,0.0,13.550,2023,21,86,False,True,0
2371,27.0,5.3,7.050,0.0,12.350,2023,21,86,False,True,0
2372,1.0,6.3,8.050,0.0,14.350,2023,21,86,True,False,0


In [3]:
name_counts = df['Name'].value_counts()

# Extract names that appear 2 or more times
names_with_multiple_occurrences = name_counts[name_counts >= 2].index

# Filter the DataFrame based on the condition
df_filtered_names = df[df['Name'].isin(names_with_multiple_occurrences)]


In [4]:
df = df_filtered_names.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 337 entries, 0 to 505
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         337 non-null    int64  
 1   D            337 non-null    float64
 2   E            337 non-null    float64
 3   ND           337 non-null    float64
 4   Total        337 non-null    float64
 5   year         337 non-null    int64  
 6   medal        337 non-null    int64  
 7   Name         337 non-null    int64  
 8   Nation       337 non-null    int64  
 9   round_final  337 non-null    bool   
 10  round_qual   337 non-null    bool   
dtypes: bool(2), float64(4), int64(5)
memory usage: 27.0 KB


In [43]:
y = df['medal']
X = df.drop(columns=['medal'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network": MLPClassifier(),
}

results = {}

# Iterate through each classifier
for name, clf in classifiers.items():
    # Create a pipeline with StandardScaler for classifiers that require it
    if name in ["SVM", "K-Nearest Neighbors", "Neural Network"]:
        clf = make_pipeline(StandardScaler(), clf)

    # Fit the model to the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    results[name] = {
        "accuracy": accuracy,
        "classification_report": report,
        "confusion_matrix": conf_matrix,
    }

# Print the results
for name, result in results.items():
    print(f"Classifier: {name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Classification Report:\n{result['classification_report']}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

Classifier: Random Forest
Accuracy: 0.9412
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.97        64
           1       0.50      0.25      0.33         4

    accuracy                           0.94        68
   macro avg       0.73      0.62      0.65        68
weighted avg       0.93      0.94      0.93        68

Confusion Matrix:
[[63  1]
 [ 3  1]]

Classifier: AdaBoost
Accuracy: 0.9706
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        64
           1       1.00      0.50      0.67         4

    accuracy                           0.97        68
   macro avg       0.98      0.75      0.83        68
weighted avg       0.97      0.97      0.97        68

Confusion Matrix:
[[64  0]
 [ 2  2]]

Classifier: SVM
Accuracy: 0.9412
Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.0

/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Classifier: AdaBoost
Accuracy: 0.9902

Classifier: Decision Tree
Accuracy: 0.9853

Classifier: Random Forest
Accuracy: 0.9853

In [7]:
X_train.head()

,Rank,D,E,ND,Total,year,Name,Nation,round_final,round_qual
128,69,5.7,7.933,0.0,13.633,2018,163,9,False,True
351,95,5.3,8.033,0.0,13.333,2019,124,67,False,True
488,10,5.6,8.250,0.0,26.900,2020,147,72,True,False
255,7,6.5,7.700,0.0,14.200,2019,218,70,True,False
298,42,5.5,8.500,0.0,14.000,2019,195,24,False,True


In [8]:
olymp.head()


,Rank,D Score,E Score,Pen.,Total,year,Name,nation,round_AAfinal,round_TeamFinal,round_TeamQual,round_final,round_qual
0,23.0,3.9,6.900,0.0,10.800,2022,236,61,False,False,False,False,True
1,23.0,3.9,6.900,0.0,10.800,2022,236,61,False,False,False,False,True
2,54.0,4.4,8.066,0.0,12.466,2023,236,61,False,False,False,False,True
3,12.0,5.8,8.400,0.0,14.200,2022,531,26,True,False,False,False,False
4,26.0,5.8,8.500,0.0,14.300,2022,531,26,False,False,False,False,True


In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=820, random_state=42)

# Fit the model to the training data
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the model's performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)

# Print the results
print("Random Forest Classifier")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Classification Report:\n{report_rf}")
print(f"Confusion Matrix:\n{conf_matrix_rf}")


Random Forest Classifier
Accuracy: 0.9647
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98        81
           1       0.67      0.50      0.57         4

    accuracy                           0.96        85
   macro avg       0.82      0.74      0.78        85
weighted avg       0.96      0.96      0.96        85

Confusion Matrix:
[[80  1]
 [ 2  2]]


In [141]:
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_pb.csv')
olymp.rename(columns={'D Score': 'D', 'E Score': 'E',
                     'Pen.': 'ND', 'nation': 'Nation'}, inplace=True)
olymp=olymp.drop(columns=['round_TeamFinal', 'round_AAfinal', 'round_TeamQual'])

In [144]:
# y_pred = clf.predict(olymp)
# y_pred = gpc.predict(olymp)
# y_pred = stacking_clf.predict(olymp)

y_pred=rf_classifier.predict(olymp)

In [145]:
ypreds  = pd.Series(y_pred)
olymp['ypred']=ypreds


In [146]:
olymp['ypred'].value_counts()

ypred
0    2366
1       8
Name: count, dtype: int64

In [147]:
filtered_df = olymp[olymp['ypred'] == 1]

In [148]:
filtered_df['Name'].unique()

array([747, 139, 552, 333])

In [149]:
name_encoded_values = [747, 139, 552, 333]

# Filter the DataFrame to get the corresponding "Name" values
result = pbnames.loc[pbnames['Name_encoded'].isin(name_encoded_values), 'Name']


In [150]:
result.unique()

array(['yul moldauer', 'curran phillips', 'noe seifert', 'jingyuan zou'],
      dtype=object)

In [151]:
pbolymp["Name"]=pbolymp["Name"].str.lower()

In [154]:
names_to_filter =['yul moldauer', 'curran phillips', 'noe seifert', 'jingyuan zou']
# Filter the DataFrame to get rows with the specified "Name" values
result = pbolymp[pbolymp['Name'].isin(names_to_filter)]


In [155]:
result['Country'].value_counts()

Country
USA    86
SUI    56
CHN    16
Name: count, dtype: int64

In [156]:
usa = result[result['Country']=='USA']

In [157]:
usa['Name'].unique()

array(['yul moldauer', 'curran phillips'], dtype=object)